In [16]:
# Convert the time stamps in midi to seconds

import pretty_midi
import numpy as np
import pandas as pd
from tqdm import tqdm

# Define the unit of observations in terms of beats
UNIT = 1/2
KEYS = ("C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B")
COMPOSITION = pd.read_csv("chroma_composition.csv")

def chord2vec(chord):
    vec = np.zeros(12)
    if ":" in chord:
        root, chroma = chord.split(":")
        root_idx = KEYS.index(root)
        composition = COMPOSITION[COMPOSITION["chroma"] == chroma]["composition"].item()
        for key_idx in composition.split():
            vec[(int(key_idx) + root_idx) % 12] = 1
    return vec

In [17]:
def extend_array(array, u = 0.5):
    r = int(1 / u)
    res = array
    for s in range(1, r):
        res = np.append(res, (array[:-1] * s * u) + (array[1:] * (r - s) * u))
    return np.sort(res)


In [28]:
for i in tqdm(range(1,910)):
    idx = f"{i:0>3}"
    midi = pd.read_csv(rf"POP909/POP909/{idx}/all_in_text_{idx}.csv", sep=",", index_col = "Unnamed: 0") 
    midi['chord'] = midi['chord'].astype(str)

    time_stamp_in_data = extend_array(np.append(np.unique(midi.beat_start), midi.tail(1).chord_end),  u  = UNIT)
    time_stamp_in_data = time_stamp_in_data[~np.isnan(time_stamp_in_data)]
    data = pd.DataFrame({'time':time_stamp_in_data}).reset_index()

    data = pd.merge(data, midi, on = 'time', how = 'outer').sort_values(by = 'time')[['index', 'time', 'mel_start', 'mel_end', 'mel_pitch', 'chord_start', 'chord_end', 'chord']].reset_index(drop = True)
    data = data.ffill()
    rows = data.time > data.mel_end
    data.loc[rows, ['mel_start', 'mel_end', 'mel_pitch']] = np.NaN
    rows = data.time > data.chord_end
    data.loc[rows, ['chord_start', 'chord_end', 'chord']] = np.NaN

    data.chord_start = np.max(np.vstack((data.time, data.chord_start)), axis = 0)
    data.chord_end = np.append(np.min(np.vstack((data.time[1:], data.chord_end[:-1])), axis = 0), data.tail(1).chord_start)


    data.mel_start = np.max(np.vstack((data.time, data.mel_start)), axis = 0)
    data.mel_end = np.append(np.min(np.vstack((data.time[1:], data.mel_end[:-1])), axis = 0), data.tail(1).mel_start)
    data = data.drop_duplicates()

    data['chord_duration'] = data.chord_end - data.chord_start
    data['mel_duration'] = data.mel_end - data.mel_start

    data.mel_pitch = (data.mel_pitch - 60) % 12

    for p in range(12):
        data[f'mel_pit_{p}'] = (data.mel_pitch == p)
        data[f'wmel_pit_{p}'] = data[f'mel_pit_{p}'] * data['mel_duration']



    by_beat = data.groupby(by=['index']).agg(chord = ("chord", "first"),
                                            mel_pit_0 = ("wmel_pit_0", "sum"),
                                            mel_pit_1 = ("wmel_pit_1", "sum"),
                                            mel_pit_2 = ("wmel_pit_2", "sum"),
                                            mel_pit_3 = ("wmel_pit_3", "sum"),
                                            mel_pit_4 = ("wmel_pit_4", "sum"),
                                            mel_pit_5 = ("wmel_pit_5", "sum"),
                                            mel_pit_6 = ("wmel_pit_6", "sum"),
                                            mel_pit_7 = ("wmel_pit_7", "sum"),
                                            mel_pit_8 = ("wmel_pit_8", "sum"),
                                            mel_pit_9 = ("wmel_pit_9", "sum"),
                                            mel_pit_10 = ("wmel_pit_10", "sum"),
                                            mel_pit_11 = ("wmel_pit_11", "sum"),
                                            )

    by_beat = by_beat.fillna(0)

    by_beat['duration'] = np.append(time_stamp_in_data[1:] - time_stamp_in_data[:-1], 0)
    for p in range(12):
        by_beat[f'mel_pit_{p}'] = by_beat[f'mel_pit_{p}'] / by_beat['duration']

    by_beat['melody'] = by_beat[['mel_pit_0', 'mel_pit_1', 'mel_pit_2', 'mel_pit_3', 'mel_pit_4',\
                                'mel_pit_5', 'mel_pit_6', 'mel_pit_7', 'mel_pit_8', 'mel_pit_9',\
                                'mel_pit_10','mel_pit_11']].values.tolist() 

    by_beat = by_beat[['chord', 'melody', 'duration']]
    by_beat['chord'] = by_beat['chord'].apply(chord2vec)
    by_beat.drop(by_beat.tail(1).index,inplace=True) 
    by_beat.to_csv(rf"POP909/POP909/{idx}/melody_chord_1_2_beat.csv")

100%|██████████| 909/909 [02:28<00:00,  6.10it/s]


In [29]:
# Add the notation by downbeats
for i in tqdm(range(1,910)):
    idx = f"{i:0>3}"

    midi = pd.read_csv(rf"POP909/POP909/{idx}/all_in_text_{idx}.csv", sep=",", index_col = "Unnamed: 0") 
    midi['chord'] = midi['chord'].astype(str)
    time_stamp_in_data = extend_array(np.append(np.unique(midi.beat_start), midi.tail(1).chord_end),  u  = UNIT)
    time_stamp_in_data = time_stamp_in_data[~np.isnan(time_stamp_in_data)]

    beat = pd.read_csv(rf"POP909/POP909/{idx}/beat_midi.txt", sep=" ", header = None) 
    beat = beat.rename(columns = {0: 'time', 1: 'downbeat1', 2: 'downbeat2'})
    beat['db_index1'] = beat['downbeat1'].cumsum()
    if beat['db_index1'][0] == 0.0:
        beat['db_index1'] = beat['db_index1'] + 1.0
    beat['db_index2'] = beat['downbeat2'].cumsum()
    if beat['db_index2'][0] == 0.0:
        beat['db_index2'] = beat['db_index2'] + 1.0

    beat = beat[['time', 'db_index1', 'db_index2']]

    data = pd.read_csv(rf"POP909/POP909/{idx}/melody_chord_1_2_beat.csv", sep=",") 
    data['time'] = time_stamp_in_data[:-1]

    data = data.merge(beat, on = 'time', how = 'left')
    data['db_index1'] = data['db_index1'].ffill()
    data['db_index2'] = data['db_index2'].ffill()

    data. to_csv(rf"POP909/POP909/{idx}/melody_chord_1_2_beat.csv")

100%|██████████| 909/909 [00:16<00:00, 56.11it/s]
